<a href="https://colab.research.google.com/github/ALIVE-ABI/alive/blob/main/GOES_ABI_LandSurfaceProduct_Downloads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import os # For Mac computers
import sys
from google.cloud import storage
import datetime
import pvlib

# Inputs


In [ ]:
amflxDirectory = "Add directory of csv file containing Ameriflux site information (ie.lat, lon, elevation, etc.)"
year_list = [2022] #Add year(s) of interest
bucket_name = 'gcp-public-data-goes-16' # Change to 'goes-17' or 'goes-18'
output_directory = '/Users/**/Downloads' # Change output directory to local path

In [ ]:
# Active Ameriflux sites (from "https://ameriflux.lbl.gov/sites/site-search/")
ameriflux_US = pd.read_csv(amflxDirectory, index_col= 'Site Id')
lats = ameriflux_US['Latitude']
lons = ameriflux_US['Longitude']
elevs = ameriflux_US['Elevation (m)']
ameriflux_US = ameriflux_US.fillna(0)

In [ ]:
#List of all variables
column_names = [
       'local_time', 'utc_time','SZA','SAA','ZenAz','doy','hour',
       'CMI_C01', 'DQF_C01', 'CMI_C02', 'DQF_C02', 'CMI_C03', 'DQF_C03',
       'BRF1', 'BRF2', 'BRF3', 'BRF_DQF',
       'LSA', 'LSA_DQF',
       'ACM', 'ACM_DQF',
       'AOD', 'AOD_DQF',
       'ADP_aero', 'ADP_smk', 'ADP_dust', 'ADP_DQF',
       'LST', 'LST_DQF',
       'DSR', 'DSR_DQF', 'NDVI','NIRv','PAR','NIRvP'
       ]

# Make an empty dictionary
amflx = {}
# Iterate through the Ameriflux sites
for index, row in ameriflux_US.iterrows():
    # Add entries to the dictionary: Ameriflux site name, new dataframe
    amflx[index] = pd.DataFrame(columns = column_names)


# Ortho functions

In [ ]:
def LonLat2ABIangle(lon_deg, lat_deg, z, H, req, rpol, e, lon_0_deg):
    '''This function finds the ABI elevation (y) and scanning (x) angles (radians) of point P,
    given a latitude and longitude (degrees)'''

    # Convert lat and lon from degrees to radians
    lon = np.radians(lon_deg)
    lat = np.radians(lat_deg)
    lon_0 = np.radians(lon_0_deg)

    # Geocentric latitude
    lat_geo = np.arctan( (rpol**2 / req**2) * np.tan(lat) )

    # Geocentric distance to point on the ellipsoid
    _rc = rpol / np.sqrt(1 - (e**2)*(np.cos(lat_geo)**2)) # this is rc if point is on the ellipsoid
    rc = _rc + z # this is rc if the point is offset from the ellipsoid by z (meters)

    # Intermediate calculations
    Sx = H - rc * np.cos(lat_geo) * np.cos(lon - lon_0)
    Sy = -rc * np.cos(lat_geo) * np.sin(lon - lon_0)
    Sz = rc * np.sin(lat_geo)

    # Calculate x and y scan angles
    y = np.arctan( Sz / Sx )
    x = np.arcsin( -Sy / np.sqrt( Sx**2 + Sy**2 + Sz**2 ) )

    return (x,y)

In [ ]:
def ABIangle2LonLat(x, y, H, req, rpol, lon_0_deg):
    '''This function finds the latitude and longitude (degrees) of point P
    given x and y, the ABI elevation and scanning angle (radians)'''

    # Intermediate calculations
    a = np.sin(x)**2 + ( np.cos(x)**2 * ( np.cos(y)**2 + ( req**2 / rpol**2 ) * np.sin(y)**2 ) )
    b = -2 * H * np.cos(x) * np.cos(y)
    c = H**2 - req**2

    rs = ( -b - np.sqrt( b**2 - 4*a*c ) ) / ( 2 * a ) # distance from satellite point (S) to P

    Sx = rs * np.cos(x) * np.cos(y)
    Sy = -rs * np.sin(x)
    Sz = rs * np.cos(x) * np.sin(y)

    # Calculate lat and lon
    lat = np.arctan( ( req**2 / rpol**2 ) * ( Sz / np.sqrt( ( H - Sx )**2 + Sy**2 ) ) )
    lat = np.degrees(lat) #*
    lon = lon_0_deg - np.degrees( np.arctan( Sy / ( H - Sx )) )

    return (lon,lat)

In [ ]:
def solar_position(lat, lon, elev):
  '''This function calculates the solar geometry throughout the day at Ameriflux site from its lat, lon and elevation'''
  date = timestamp.pd_asdatetime()
  solar_pos = pvlib.solarposition.get_solarposition(time, lat, lon, elev)
  SZA = solar_pos['zenith'].reset_index()
  az = solar_pos['azimuth'].reset_index()
  zenAz = az + SZA
  df = pd.concat([df, zenith], axis=1)

# Download Functions


In [ ]:
def make_abi_timeseries(filename, product, unique_df):
    '''Given a directory of GOES ABI products, the function creates a timeseries of data variables (specified in data_vars) for a single point (at lon, lat, elevation).
	   Returns a pandas dataframe and  outputs a csv file for each year (optionally, for each day).'''

    # Make an empty dictionary for each site
    unique_df = {}

    # List the relevant variables for each product
    product = product
    if product == 'MCMIPF':
      data_vars =     ['CMI_C01', 'DQF_C01', 'CMI_C02', 'DQF_C02', 'CMI_C03', 'DQF_C03']
      column_names =  ['CMI_C01', 'DQF_C01', 'CMI_C02', 'DQF_C02', 'CMI_C03', 'DQF_C03']
    if product == 'BRFF':
      data_vars =     ['BRF1', 'BRF2', 'BRF3', 'DQF']
      column_names =  ['BRF1', 'BRF2', 'BRF3', 'BRF_DQF']
    if product == 'LSAF':
      data_vars =     ['LSA', 'DQF']
      column_names =  ['LSA', 'LSA_DQF']
    if product == 'ACMF':
      data_vars =     ['BCM', 'DQF']
      column_names =  ['ACM', 'ACM_DQF']
    if product == 'AODF':
      data_vars =     ['AOD', 'DQF']
      column_names =  ['AOD', 'AOD_DQF']
    if product == 'ADPF':
      data_vars =     ['Aerosol', 'Smoke', 'Dust','DQF']
      column_names =  ['ADP_aero', 'ADP_smk', 'ADP_dust', 'ADP_DQF']
    if product == 'LSTF':
      data_vars =     ['LST', 'DQF']
      column_names =  ['LST', 'LST_DQF']
    if product == 'DSRF':
      data_vars =     ['DSR', 'DQF']
      column_names =  ['DSR', 'DSR_DQF']

    print('New file:' + product)
    f = xr.open_dataset(filename, decode_times=False)\

    # Access each Ameriflux Site dataframe by iterating through each dictionary item
    for index, df in amflx.items():
      site_df = pd.DataFrame(columns = [])

      # For each Ameriflux site, set the lat, lon and elevation by referring to entries in the US Ameriflux Sites database
      site_lat = ameriflux_US.loc[str(index), 'Latitude']
      site_lon = ameriflux_US.loc[str(index), 'Longitude']
      site_elev = ameriflux_US.loc[str(index), 'Elevation (m)']

      # Read goes_imager_projection values needed for geometry calculations
      # and compute the corresponding look angles (in radiance) for the lat, lon, elevation we are interested in
      x_rad, y_rad = LonLat2ABIangle(site_lon,
                                   site_lat,
                                   site_elev,
                                   f.goes_imager_projection.perspective_point_height + f.goes_imager_projection.semi_major_axis,
                                   f.goes_imager_projection.semi_major_axis,
                                   f.goes_imager_projection.semi_minor_axis,
                                   0.0818191910435, # GRS-80 eccentricity
                                   f.goes_imager_projection.longitude_of_projection_origin)

      # Get the timestamp (UTC) for this observation
      timestamp = pd.Timestamp(f.time_coverage_start).replace(tzinfo=None)

      # Create an empty dictionary we will populate with values from file f
      this_row_dict = {}

      # Create an empty list of the same length as data_vars to hold each variable's value
      # and create a matching empty list to hold each column's name in the matching
      values = ['' for v in data_vars]
      column = ['' for v in data_vars]

      # For each variable we are interested, specified in the list "data_vars"
      # Create an empty dataframe
      one_site =  pd.DataFrame(columns=[])

      for i, var in enumerate(data_vars):
        # Find corresponding pixel data_var value nearest to these scan angles y_rad and x_rad
        values[i] = f[var].sel(y=y_rad, x=x_rad, method='nearest').values.mean()
        column[i] = column_names[i]

        # Create a dictionary for this row of values (where each row is a GOES-R observation time)
        this_row_dict = dict(zip(column, values ))
        this_row_dict['timestamp'] = timestamp

        # Concatenate the half-hourly measurements for each hour
        one_site = pd.concat([one_site, pd.DataFrame([this_row_dict])], ignore_index=True)

      # Drop duplicates if there are any, keep the last one
      one_site = one_site.drop_duplicates(['timestamp'], keep='last', inplace=False)

      # Set the dataframe intext to the timestamp column
      one_site = one_site.set_index('timestamp', inplace = False, verify_integrity = False)

      # Store two half-hourly observations at the site in a dictionary
      unique_df[str(index)] = one_site

    return unique_df

In [ ]:
def DSR_make_abi_timeseries(filename, product, unique_df):
    '''Given a directory of GOES ABI products, create a timeseries of data variables (specified in data_vars) for a single point (at lon, lat, elevation).
	   Returns a pandas dataframe, optional output to a csv file.'''

    unique_df = {}

    #List the relevant variables for each product
    product = product
    if product == 'DSRF':
      data_vars =     ['DSR', 'DQF']
      column_names =  ['DSR', 'DSR_DQF']

    #print('Creating a timeseries of each of these variables {variable} from {product}'.format(variable=data_vars, product = product))
    print('New file:' + product)
    f = xr.open_dataset(filename)

    #Access each Ameriflux Site dataframe by iterating through each dictionary item
    for index, df in amflx.items():
      #For each Ameriflux site, set the lat, lon and elevation by referring to entries in the US Ameriflux Sites database
      site_lat = ameriflux_US.loc[str(index), 'Latitude']
      site_lon = ameriflux_US.loc[str(index), 'Longitude']
      site_elev = ameriflux_US.loc[str(index), 'Elevation (m)']

      # Read goes_lat_lon_projection values needed for geometry calculations
      # and compute the corresponding look angles (in radiance) for the lat, lon, elevation we are interested in
      x_rad, y_rad = LonLat2ABIangle(site_lon,
                                     site_lat,
                                     site_elev,
                                     35786023.0 + f.goes_lat_lon_projection.semi_major_axis, # DSR missing metadata, setting perspective point height = 35786023.0
                                     f.goes_lat_lon_projection.semi_major_axis,
                                     f.goes_lat_lon_projection.semi_minor_axis,
                                     0.0818191910435, # GRS-80 eccentricity
                                     -75.0) # DSR missing longitude of projection origin, set to -75.0

      lat, lon = ABIangle2LonLat(x_rad,
                                 y_rad,
                                 35786023.0 + f.goes_lat_lon_projection.semi_major_axis,
                                 f.goes_lat_lon_projection.semi_major_axis,
                                 f.goes_lat_lon_projection.semi_minor_axis,
                                 -75.0)

      # Get the timestamp for this observation (UTC)
      timestamp = pd.Timestamp(f.time_coverage_start).replace(tzinfo=None)

      # Create an empty dictionary we will populate with values from file f
      this_row_dict = {}

      # Create an empty list of the same length as data_vars to hold each variable's value
      # and create a matching empty list to hold each column's name in the matching
      values = ['' for v in data_vars]
      column = ['' for v in data_vars]

      # For each variable we are interested, specified in the list "data_vars"
      one_site =  pd.DataFrame(columns=[])

      for i, var in enumerate(data_vars):
        # Find corresponding pixel data_var value nearest to the lat/lon
        values[i] = f[var].sel(lat= lat, lon= lon, method='nearest').values.mean()
        column[i] = column_names[i]

        # Create a dictionary for this row of values (where each row is a GOES-R observation time)
        this_row_dict = dict(zip(column, values ))
        this_row_dict['timestamp'] = timestamp

        one_site = pd.concat([one_site, pd.DataFrame([this_row_dict])], ignore_index=True)

      # Drop duplicates if there are any, keep the first one
      one_site = one_site.drop_duplicates(['timestamp'], keep='last', inplace=False)

      # Set the dataframe intext to the timestamp column
      one_site = one_site.set_index('timestamp', inplace = False, verify_integrity = False)

      # Store one hourly observation at the site in a dictionary
      unique_df[str(index)] = one_site

    return unique_df

In [ ]:
def download_blobs(bucket_name, file_prefix, directory, product, unique_df):
    """Lists all the blobs in the bucket."""

    # Create anonymous Google Cloud storage credentials to access public buckets
    storage_client = storage.Client.create_anonymous_client()

    # Access GOES-R bucket for specific date/time/product and list its content
    bucket = storage_client.get_bucket(bucket_name)
    blobs = storage_client.list_blobs(bucket_name, prefix = file_prefix)
    blob_list = list(blobs)

    # For land surface temperature (LST)
    if product == 'LSTF':
        one_blob = blob_list[0]
        location, blob_filename= os.path.split(one_blob.name)
        destination_file_name = directory + blob_filename
        eachBlob = storage.Blob(one_blob.name, bucket)
        # Download LST file to local destination
        file = eachBlob.download_to_filename(destination_file_name)
        # Make and store single hourly LST observation
        one_df = make_abi_timeseries(destination_file_name, product, unique_df)
        os.remove(destination_file_name)

    # For downward shortwave radiation (DSR)
    elif product == 'DSRF':
        one_blob = blob_list[0]
        location, blob_filename= os.path.split(one_blob.name)
        destination_file_name = directory + blob_filename
        eachBlob = storage.Blob(one_blob.name, bucket)
        # Download DSR file to local destination
        file = eachBlob.download_to_filename(destination_file_name)
        # Make and store single hourly DSR observation
        one_df = DSR_make_abi_timeseries(destination_file_name, product, unique_df)
        os.remove(destination_file_name)

    # For all other ABI L2 products (with refresh rates > once/hour)
    else:
        blob_dict = {}
        one_df = {}
        # Get files 0 and 3 from the list (around 10 and 30 mins past the hour)
        halfHour_blobs = [blob_list[0],blob_list[3]]
        # Loop through both files
        for blob in halfHour_blobs:
            location, blob_filename= os.path.split(blob.name)
            destination_file_name = directory + blob_filename
            eachBlob = storage.Blob(blob.name, bucket)
            # Download each file to local destination
            file = eachBlob.download_to_filename(destination_file_name)
            # Make and store single half-hourly observation
            data_table = make_abi_timeseries(destination_file_name, product, unique_df)
            blob_dict[blob] = data_table
            os.remove(destination_file_name)
        # Combine two half hourly observations into one dataframe
        for index, row in ameriflux_US.iterrows():
            one_table = pd.concat([blob_dict[blob_list[0]][index],
                                   blob_dict[blob_list[3]][index]],
                                )
            one_df[index] = one_table
    return one_df

In [ ]:
def concat_days(index,day_dict) :
    "This function concatenates each daily dataframe into one yearly dataframe. Add day 366 for Leap Years."
    one_year = pd.concat([
    day_dict['001'][index], day_dict['002'][index], day_dict['003'][index], day_dict['004'][index], day_dict['005'][index],
    day_dict['006'][index], day_dict['007'][index], day_dict['008'][index], day_dict['009'][index], day_dict['010'][index],
    day_dict['011'][index], day_dict['012'][index], day_dict['013'][index], day_dict['014'][index], day_dict['015'][index],
    day_dict['016'][index], day_dict['017'][index], day_dict['018'][index], day_dict['019'][index], day_dict['020'][index],
    day_dict['021'][index], day_dict['022'][index], day_dict['023'][index], day_dict['024'][index], day_dict['025'][index],
    day_dict['026'][index], day_dict['027'][index], day_dict['028'][index], day_dict['029'][index], day_dict['030'][index],
    day_dict['031'][index], day_dict['032'][index], day_dict['033'][index], day_dict['034'][index], day_dict['035'][index],
    day_dict['036'][index], day_dict['037'][index], day_dict['038'][index], day_dict['039'][index], day_dict['040'][index],
    day_dict['041'][index], day_dict['042'][index], day_dict['043'][index], day_dict['044'][index], day_dict['045'][index],
    day_dict['046'][index], day_dict['047'][index], day_dict['048'][index], day_dict['049'][index], day_dict['050'][index],
    day_dict['051'][index], day_dict['052'][index], day_dict['053'][index], day_dict['054'][index], day_dict['055'][index],
    day_dict['056'][index], day_dict['057'][index], day_dict['058'][index], day_dict['059'][index], day_dict['060'][index],
    day_dict['061'][index], day_dict['062'][index], day_dict['063'][index], day_dict['064'][index], day_dict['065'][index],
    day_dict['066'][index], day_dict['067'][index], day_dict['068'][index], day_dict['069'][index], day_dict['070'][index],
    day_dict['071'][index], day_dict['072'][index], day_dict['073'][index], day_dict['074'][index], day_dict['075'][index],
    day_dict['076'][index], day_dict['077'][index], day_dict['078'][index], day_dict['079'][index], day_dict['080'][index],
    day_dict['081'][index], day_dict['082'][index], day_dict['083'][index], day_dict['084'][index], day_dict['085'][index],
    day_dict['086'][index], day_dict['087'][index], day_dict['088'][index], day_dict['089'][index], day_dict['090'][index],
    day_dict['091'][index], day_dict['092'][index], day_dict['093'][index], day_dict['094'][index], day_dict['095'][index],
    day_dict['096'][index], day_dict['097'][index], day_dict['098'][index], day_dict['099'][index], day_dict['100'][index],
    day_dict['101'][index], day_dict['102'][index], day_dict['103'][index], day_dict['104'][index], day_dict['105'][index],
    day_dict['106'][index], day_dict['107'][index], day_dict['108'][index], day_dict['109'][index], day_dict['110'][index],
    day_dict['111'][index], day_dict['112'][index], day_dict['113'][index], day_dict['114'][index], day_dict['115'][index],
    day_dict['116'][index], day_dict['117'][index], day_dict['118'][index], day_dict['119'][index], day_dict['120'][index],
    day_dict['121'][index], day_dict['122'][index], day_dict['123'][index], day_dict['124'][index], day_dict['125'][index],
    day_dict['126'][index], day_dict['127'][index], day_dict['128'][index], day_dict['129'][index], day_dict['130'][index],
    day_dict['131'][index], day_dict['132'][index], day_dict['133'][index], day_dict['134'][index], day_dict['135'][index],
    day_dict['136'][index], day_dict['137'][index], day_dict['138'][index], day_dict['139'][index], day_dict['140'][index],
    day_dict['141'][index], day_dict['142'][index], day_dict['143'][index], day_dict['144'][index], day_dict['145'][index],
    day_dict['146'][index], day_dict['147'][index], day_dict['148'][index], day_dict['149'][index], day_dict['150'][index],
    day_dict['151'][index], day_dict['152'][index], day_dict['153'][index], day_dict['154'][index], day_dict['155'][index],
    day_dict['156'][index], day_dict['157'][index], day_dict['158'][index], day_dict['159'][index], day_dict['160'][index],
    day_dict['161'][index], day_dict['162'][index], day_dict['163'][index], day_dict['164'][index], day_dict['165'][index],
    day_dict['166'][index], day_dict['167'][index], day_dict['168'][index], day_dict['169'][index], day_dict['170'][index],
    day_dict['171'][index], day_dict['172'][index], day_dict['173'][index], day_dict['174'][index], day_dict['175'][index],
    day_dict['176'][index], day_dict['177'][index], day_dict['178'][index], day_dict['179'][index], day_dict['180'][index],
    day_dict['181'][index], day_dict['182'][index], day_dict['183'][index], day_dict['184'][index], day_dict['185'][index],
    day_dict['186'][index], day_dict['187'][index], day_dict['188'][index], day_dict['189'][index], day_dict['190'][index],
    day_dict['191'][index], day_dict['192'][index], day_dict['193'][index], day_dict['194'][index], day_dict['195'][index],
    day_dict['196'][index], day_dict['197'][index], day_dict['198'][index], day_dict['199'][index], day_dict['200'][index],
    day_dict['201'][index], day_dict['202'][index], day_dict['203'][index], day_dict['204'][index], day_dict['205'][index],
    day_dict['206'][index], day_dict['207'][index], day_dict['208'][index], day_dict['209'][index], day_dict['210'][index],
    day_dict['211'][index], day_dict['212'][index], day_dict['213'][index], day_dict['214'][index], day_dict['215'][index],
    day_dict['216'][index], day_dict['217'][index], day_dict['218'][index], day_dict['219'][index], day_dict['220'][index],
    day_dict['221'][index], day_dict['222'][index], day_dict['223'][index], day_dict['224'][index], day_dict['225'][index],
    day_dict['226'][index], day_dict['227'][index], day_dict['228'][index], day_dict['229'][index], day_dict['230'][index],
    day_dict['231'][index], day_dict['232'][index], day_dict['233'][index], day_dict['234'][index], day_dict['235'][index],
    day_dict['236'][index], day_dict['237'][index], day_dict['238'][index], day_dict['239'][index], day_dict['240'][index],
    day_dict['241'][index], day_dict['242'][index], day_dict['243'][index], day_dict['244'][index], day_dict['245'][index],
    day_dict['246'][index], day_dict['247'][index], day_dict['248'][index], day_dict['249'][index], day_dict['250'][index],
    day_dict['251'][index], day_dict['252'][index], day_dict['253'][index], day_dict['254'][index], day_dict['255'][index],
    day_dict['256'][index], day_dict['257'][index], day_dict['258'][index], day_dict['259'][index], day_dict['260'][index],
    day_dict['261'][index], day_dict['262'][index], day_dict['263'][index], day_dict['264'][index], day_dict['265'][index],
    day_dict['266'][index], day_dict['267'][index], day_dict['268'][index], day_dict['269'][index], day_dict['270'][index],
    day_dict['271'][index], day_dict['272'][index], day_dict['273'][index], day_dict['274'][index], day_dict['275'][index],
    day_dict['276'][index], day_dict['277'][index], day_dict['278'][index], day_dict['279'][index], day_dict['280'][index],
    day_dict['281'][index], day_dict['282'][index], day_dict['283'][index], day_dict['284'][index], day_dict['285'][index],
    day_dict['286'][index], day_dict['287'][index], day_dict['288'][index], day_dict['289'][index], day_dict['290'][index],
    day_dict['291'][index], day_dict['292'][index], day_dict['293'][index], day_dict['294'][index], day_dict['295'][index],
    day_dict['296'][index], day_dict['297'][index], day_dict['298'][index], day_dict['299'][index], day_dict['300'][index],
    day_dict['301'][index], day_dict['302'][index], day_dict['303'][index], day_dict['304'][index], day_dict['305'][index],
    day_dict['306'][index], day_dict['307'][index], day_dict['308'][index], day_dict['309'][index], day_dict['310'][index],
    day_dict['311'][index], day_dict['312'][index], day_dict['313'][index], day_dict['314'][index], day_dict['315'][index],
    day_dict['316'][index], day_dict['317'][index], day_dict['318'][index], day_dict['319'][index], day_dict['320'][index],
    day_dict['321'][index], day_dict['322'][index], day_dict['323'][index], day_dict['324'][index], day_dict['325'][index],
    day_dict['326'][index], day_dict['327'][index], day_dict['328'][index], day_dict['329'][index], day_dict['330'][index],
    day_dict['331'][index], day_dict['332'][index], day_dict['333'][index], day_dict['334'][index], day_dict['335'][index],
    day_dict['336'][index], day_dict['337'][index], day_dict['338'][index], day_dict['339'][index], day_dict['340'][index],
    day_dict['341'][index], day_dict['342'][index], day_dict['343'][index], day_dict['344'][index], day_dict['345'][index],
    day_dict['346'][index], day_dict['347'][index], day_dict['348'][index], day_dict['349'][index], day_dict['350'][index],
    day_dict['351'][index], day_dict['352'][index], day_dict['353'][index], day_dict['354'][index], day_dict['355'][index],
    day_dict['356'][index], day_dict['357'][index], day_dict['358'][index], day_dict['359'][index], day_dict['360'][index],
    day_dict['361'][index], day_dict['362'][index], day_dict['363'][index], day_dict['364'][index], day_dict['365'][index],
    ])
    return one_year


# Run functions

In [ ]:
# Create empty dictionaries
variable_dict = {}
unique_df = {}
day_dict = {}
each_day_dict = {}
hour_dict = {}

# Loop through each year
for year in year_list: #Year or list of years
  year = year

  # Loop through each day
  for day in range(1,366): #Change for leap year
    day = day
    day = str(day).zfill(3)
    print("NEXT DAY! Day = " + str(day))

    # Loop through each hour
    for hour in range(0,24):
        hour = hour
        hour = str(hour).zfill(2)
        print("Next hour! Hour = " + str(hour))
        file_list = []

        # Loop through each product
        for product in ['MCMIPF', 'BRFF', 'LSAF', 'ACMF', 'AODF', 'ADPF', 'LSTF', 'DSRF']:
            product = product
            folder_path = 'ABI-L2-{product}/{year}/{day}/{hour}/'.format(product = product,
                                                                     year = year,
                                                                     day = day,
                                                                     hour = hour)
            short_folder_path = 'ABI-L2-{product}/{year}/{day}/'.format(product = product,
                                                                     year = year,
                                                                     day = day)
            local_folder = local_path + short_folder_path
            os.makedirs(local_folder, exist_ok=True)

            # Download GOES-R ABI product file to local directory for temporary storage
            each_var = download_blobs(bucket_name, folder_path, local_folder, product, unique_df)

            #Store each dataframe of each product observation in dictionary
            variable_dict[str(product)] = each_var

        #Iterate through the Ameriflux sites
        site_dict = {}
        for index, row in ameriflux_US.iterrows():

            # Concatenate observations from each ABI product into a single dataframe with half-hourly timesteps
            all_var = pd.concat([variable_dict['MCMIPF'][index],
                                 variable_dict['BRFF'][index],
                                 variable_dict['LSAF'][index],
                                 variable_dict['ACMF'][index],
                                 variable_dict['AODF'][index],
                                 variable_dict['ADPF'][index],
                                 variable_dict['LSTF'][index],
                                 variable_dict['DSRF'][index],
                                 ], axis=1, join='outer')
            # Calculate indices 'NDVI' and 'NIRv' from surface reflectances
            BRF1 = all_var['BRF1']
            BRF2 = all_var['BRF2']
            BRF3 = all_var['BRF3']
            all_var['NDVI'] = (BRF3 - BRF2)/(BRF3 + BRF2)
            all_var['NIRv'] = BRF3 * all_var['NDVI']

            # Estimate PAR and NIRv from DSR
            all_var['PAR'] = 0.45 * all_var['DSR'] # PAR in W/m2
            all_var['NIRvP'] = all_var['NIRv'] * all_var['PAR']

            #Manage UTC timestamps, add local time, and add other datetime variables
            all_var = all_var.reset_index()
            all_var['UTC_TIME'] = pd.to_datetime(all_var.timestamp)
            all_var = all_var.set_index('timestamp')
            hr_offset = float(ameriflux_US.loc[str(index), 'Timezone/UTC offset'].split('+')[1])
            all_var['LOCAL_TIME'] = all_var['utc_time'] - datetime.timedelta(hours=hr_offset)
            all_var['DOY'] = all_var.local_time.dt.dayofyear
            all_var['HOUR'] = all_var.local_time.dt.hour

            # Calculate solar position using pvlib module
            solar_pos = pvlib.solarposition.get_solarposition(all_var['utc_time'],
                                                              ameriflux_US.loc[str(index), 'Latitude'],
                                                              ameriflux_US.loc[str(index), 'Longitude'],
                                                              ameriflux_US.loc[str(index), 'Elevation (m)'])
            all_var['SZA'] = solar_pos['zenith']
            all_var['SAA'] = solar_pos['azimuth']
            # Add solar zenith angle and solar azimuth angle to get unique solar position
            all_var['SOLAR_POS'] = all_var['SZA'] + all_var['SAA']

            # Add hourly dataframe for all products at site to a site dictionary
            site_dict[index] = all_var

        # Add hourly site dictionaries to an hourly dictionary
        hour_dict[str(hour)] = site_dict

    each_day_dict = {}
    # Iterate through each hour of the day
    for index, row in ameriflux_US.iterrows():
      # Concatenate dataframes from each hour of the day at the site (stored in hourly dictionary)
      one_day = pd.concat(  [hour_dict['00'][index], hour_dict['01'][index], hour_dict['02'][index],
                             hour_dict['03'][index], hour_dict['04'][index], hour_dict['05'][index],
                             hour_dict['06'][index], hour_dict['07'][index], hour_dict['08'][index],
                             hour_dict['09'][index], hour_dict['10'][index], hour_dict['11'][index],
                             hour_dict['12'][index], hour_dict['13'][index], hour_dict['14'][index],
                             hour_dict['15'][index], hour_dict['16'][index], hour_dict['17'][index],
                             hour_dict['18'][index], hour_dict['19'][index], hour_dict['20'][index],
                             hour_dict['21'][index], hour_dict['22'][index], hour_dict['23'][index],
                             ])
      # Interpolate half-hour observations (using cubic spline interpolation) between hourly DSR and LST observations
      if np.sum(one_day['LST'].count()) > 1:
        one_day['LST'] = one_day['LST'].interpolate(method='cubicspline', limit = 1)
        one_day['LST_DQF'] = one_day['LST_DQF'].fillna(method="ffill")
      if np.sum(one_day['DSR'].count()) > 1:
        one_day['DSR'] = one_day['DSR'].interpolate(method='cubicspline', limit = 1)
        one_day['DSR_DQF'] = one_day['DSR_DQF'].fillna(method="ffill")

      # Uncomment below to download files daily and write-over previous days
      #csv_file = one_day.to_csv('/<<local path>>/'+ str(index) + '.csv', columns = column_names)

      # Store each day dataframe for site in a daily dictionary
      each_day_dict[index] = one_day

    # Store each daily datafram eover the year in day dictionary
    day_dict[day] = each_day_dict
    print("Day complete")

  each_year_dict = {}

  # Iterate through each Ameriflux site
  for index, row in ameriflux_US.iterrows():

      # Concatenate dataframes from each day of the year at the site (stored in day dictionary)
      one_year = concat_days(index, day_dict)

      # Convert pandas dataframe to csv file for each year and sotre at local directory
      csv_file = one_year.to_csv(local_path + '/amflx_data/byYear/'+ str(index) + '.csv', columns = column_names)
      each_year_dict[index] = one_year
  print("Year complete")

  #For multiple years, concatenate dataframe from each_year_dict